In [2]:
# python 3.9.8

In [3]:
%pip install numpy==1.26.0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install onnx

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import onnx
import onnxruntime as ort
from onnxruntime_extensions import get_library_path
from os import cpu_count

sentences = ["hello world"]

def load_onnx_model(model_filepath):
  _options = ort.SessionOptions()
  _options.inter_op_num_threads, _options.intra_op_num_threads = cpu_count(), cpu_count()
  _options.register_custom_ops_library(get_library_path())
  _providers = ["CPUExecutionProvider"]  # could use ort.get_available_providers()
  return ort.InferenceSession(path_or_bytes=model_filepath, sess_options=_options, providers=_providers)

model = load_onnx_model("/home/Gokulprasath.R/Downloads/model (2).onnx")

model_outputs = model.run(output_names=["outputs"], input_feed={"inputs": sentences})[0]
print(model_outputs)


[[-4.65003364e-02  4.96842936e-02 -1.67709589e-02 -1.37947220e-02
   1.14190876e-02  5.73208444e-02  1.81501068e-03  1.34836091e-02
  -1.29638342e-02 -9.15626995e-03  4.52919602e-02  9.73468239e-04
  -1.64914057e-02 -1.16463006e-02 -8.16036761e-02  3.03941853e-02
  -8.69829208e-03 -4.86878753e-02 -2.23871786e-02 -3.73383835e-02
  -5.04437648e-02 -2.55398098e-02 -4.53855544e-02 -1.51968207e-02
   2.16337908e-02 -6.55875774e-04 -1.06775768e-01 -4.10663374e-02
  -1.30895339e-02  3.71572152e-02 -1.69861186e-02 -2.06289329e-02
  -2.03848723e-02 -9.04931780e-03  6.20940607e-03 -3.66252214e-02
  -8.84809718e-03 -4.39511091e-02 -6.48350455e-03 -7.44510815e-02
   2.50734825e-04 -1.57219209e-02  3.87140154e-03  8.21539983e-02
   6.30682781e-02  7.26114085e-04 -2.79770400e-02 -1.12528522e-02
   1.06348367e-02 -1.80301964e-02  3.09963785e-02 -2.16878671e-02
  -4.38600332e-02  4.85523902e-02 -1.48778595e-02  1.64098460e-02
   5.14517725e-02  3.43276076e-02  8.14242885e-02  1.11994771e-02
  -4.94876

In [8]:
%pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.4/251.4 kB 1.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import onnxruntime as rt
from onnxruntime_extensions import get_library_path
from os import cpu_count

In [3]:
# Load the ONNX model
def load_onnx_model(model_filepath):
    _options = rt.SessionOptions()
    _options.inter_op_num_threads = cpu_count()
    _options.intra_op_num_threads = cpu_count()
    _options.register_custom_ops_library(get_library_path())
    _providers = ["CPUExecutionProvider"]
    return rt.InferenceSession(path_or_bytes=model_filepath, sess_options=_options, providers=_providers)

model_filepath = "/home/Gokulprasath.R/Downloads/model (2).onnx"
model = load_onnx_model(model_filepath)
print("Model loaded successfully with custom ops")

Model loaded successfully with custom ops


In [4]:
# Generate embeddings
def get_embeddings(model, utterances):
    inputs = model.get_inputs()
    input_name = inputs[0].name
    
    embeddings = []
    for utterance in utterances:
        # Prepare input for the model
        input_data = np.array([utterance], dtype=object)
        # Get the embedding
        embedding = model.run(None, {input_name: input_data})[0]
        embeddings.append(embedding)
    return np.array(embeddings).squeeze(axis=1)

In [5]:
# Compute cosine similarity
def find_similarities(test_embedding, train_embeddings, threshold=0.6):
    similarities = cosine_similarity(test_embedding.reshape(1, -1), train_embeddings)
    similarities = similarities.flatten()

    matched_indices = np.where(similarities >= threshold)[0]
    print([(index, similarities[index], True) for index in matched_indices])
    
    if len(matched_indices) == 0:
        max_index = np.argmax(similarities)
        return [(max_index, similarities[max_index], False)]
    else:
        return [(index, similarities[index], True) for index in matched_indices]

In [6]:

# Load data from the Excel file
df_train = pd.read_excel("utterances.xlsx", sheet_name="Training Utterances")
df_test = pd.read_excel("utterances.xlsx", sheet_name="Testing Utterances")

# Generate embeddings
train_utterances = df_train["Utterance"].tolist()
test_utterances = df_test["Utterance"].tolist()

train_embeddings = get_embeddings(model, train_utterances)
test_embeddings = get_embeddings(model, test_utterances)

In [8]:
# Select one testing utterance for comparison
results = []
for selected_test_index in range(len(test_utterances)):
    selected_test_embedding = test_embeddings[selected_test_index]
    test_utterance = test_utterances[selected_test_index]
    test_intent = df_test.loc[selected_test_index, "Intent"]
    test_id = df_test.loc[selected_test_index, "ID"]

    # Find similarities
    similarities = find_similarities(selected_test_embedding, train_embeddings)

    for index, similarity, is_above_threshold in similarities:
        matched_utterance = train_utterances[index]
        matched_intent = df_train.loc[index, "Intent"]
        matched_id = df_train.loc[index, "ID"]
        
        if is_above_threshold:
            results.append([test_utterance, test_intent, test_id, matched_id, matched_intent, matched_utterance])
        else:
            results.append([test_utterance, test_intent, test_id, matched_id, matched_intent, f" - {matched_utterance}"])

# Create DataFrame for results
df_results = pd.DataFrame(results, columns=["Testing utterance", "Expected Intent", "Expected ID", "Matched ID", "Matched Intent", "Matched utterance"])

# Save results to CSV
df_results.to_csv("similarity_results.csv", index=False)

# Output a sample of the results
print(df_results.head())

[(1, 1.0000004, True), (2, 0.66838694, True), (3, 0.79187524, True), (5, 0.7598724, True), (6, 0.8279598, True), (9, 0.6089289, True)]
[(11, 0.8393193, True), (12, 0.83791685, True), (13, 0.6136917, True), (15, 0.64152414, True), (18, 0.77708215, True), (19, 0.70270336, True)]
[]
[(30, 0.83342093, True), (31, 0.7773285, True)]
[(1, 0.729414, True), (3, 0.6768036, True), (4, 0.61927795, True), (5, 0.6542609, True), (6, 0.60675144, True), (9, 0.6387447, True)]
[(11, 0.704901, True), (15, 0.60087, True), (18, 0.6313546, True)]
[(21, 0.63171935, True), (28, 0.60464996, True)]
[(30, 0.80920047, True), (31, 0.8195977, True)]
[(1, 0.9497502, True), (2, 0.6683379, True), (3, 0.8774365, True), (5, 0.7598709, True), (6, 0.81508744, True), (7, 0.6322825, True), (9, 0.69104147, True)]
[(11, 0.84097826, True), (12, 0.7021097, True), (13, 0.70678705, True), (15, 0.71619725, True), (17, 0.6624557, True), (19, 0.7987547, True)]
                    Testing utterance Expected Intent Expected ID Matched 